In [1]:
# Wormy (a Nibbles clone)
# By Al Sweigart al@inventwithpython.com
# http://inventwithpython.com/pygame
# Released under a "Simplified BSD" license

import random, pygame, sys, time
from pygame.locals import *

FPS = 15
WINDOWWIDTH = 640
WINDOWHEIGHT = 480
CELLSIZE = 20
assert WINDOWWIDTH % CELLSIZE == 0, "Window width must be a multiple of cell size."
assert WINDOWHEIGHT % CELLSIZE == 0, "Window height must be a multiple of cell size."
CELLWIDTH = int(WINDOWWIDTH / CELLSIZE)
CELLHEIGHT = int(WINDOWHEIGHT / CELLSIZE)

#             R    G    B
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
DARKGREEN = (0, 155, 0)
YELLOW = (255, 255, 0)
DARKGRAY = (40, 40, 40)
GRAY = (185, 185, 185)
BGCOLOR = BLACK

TEXTSHADOWCOLOR = GRAY
TEXTCOLOR = WHITE

UP = 'up'
DOWN = 'down'
LEFT = 'left'
RIGHT = 'right'

HEAD = 0  # 지렁이의 머리의 index는 0으로 고정.

def main():  # main 부분
    global FPSCLOCK, DISPLAYSURF, BASICFONT, BIGFONT
    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    BASICFONT = pygame.font.Font('D2Coding.ttc', 18)
    BIGFONT = pygame.font.Font('D2Coding.ttc', 100)
    pygame.display.set_caption('2019074075_JaehoCho')  # 게임 상태창
    # 본 게임 시작 전 게임 시작 화면을 보여준다.
    showStartScreen()
    # 실제 게임을 실행 시키는 함수를 호출하는 while loop이다.
    # 우선 0 ~ 2의 난수를 생성. 이를 바탕으로 노래 파일을 실행시킨다.
    # runGame() 함수를 실행시킨다. 이는 실제 게임 실행 함수이다.
    # runGame()에서 game over되면 showGameOverScreen()을 실행한다.
    # 게임을 종료 시키지 않는 한 이 while loop는 무한 반복하게 된다.
    while True:
        num = random.randint(0, 2)  # 0,1,2중에서 난수 생성
        if num == 0:
            pygame.mixer.music.load('Hover.mp3')
        elif num == 1:
            pygame.mixer.music.load('Our_Lives_Past.mp3')
        else:
            pygame.mixer.music.load('Platform_9.mp3')
        pygame.mixer.music.play(-1, 0.0)
        runGame()  # 게임 실행
        showGameOverScreen()  # 게임 오버하면 실행하는 함수

# 실제 게임을 실행 시키는 함수.
def runGame():
    # 게임 시작 시 start_time에 시작 시각을 기록한다.
    start_time = time.time()
    # 지렁이의 시작점을 임의로 지정한다.
    startx = random.randint(5, CELLWIDTH - 6)
    starty = random.randint(5, CELLHEIGHT - 6)
    # 지렁이 형성. 이 때 지렁이의 머리는 wormCoords[0]이며 따라서 HEAD 상수를 위에 0으로 선언 했다.
    wormCoords = [{'x': startx, 'y': starty},
                  {'x': startx - 1, 'y': starty},
                  {'x': startx - 2, 'y': starty}]
    direction = RIGHT # 이는 지렁이가 시작 시 오른쪽으로 움직인다는 것을 의미한다.

    # 시작 시 사과를 random한 위치에 배치한다.
    apple = getRandomLocation()
    while True:  # 게임 루프
        for event in pygame.event.get():  # 키보드 입력 시 입력 받은 값을 event에 in.
            if event.type == QUIT:
                terminate()
            elif event.type == KEYDOWN:
                # 상하좌우 KEYDOWN 이벤트에 대해서 지렁이의 움직임 방향을 결정한다.
                # w, s, a, d 역시 각 상하좌우에 대응한다.
                if (event.key == K_LEFT or event.key == K_a) and direction != RIGHT:
                    direction = LEFT
                elif (event.key == K_RIGHT or event.key == K_d) and direction != LEFT:
                    direction = RIGHT
                elif (event.key == K_UP or event.key == K_w) and direction != DOWN:
                    direction = UP
                elif (event.key == K_DOWN or event.key == K_s) and direction != UP:
                    direction = DOWN
                # 게임 중 esc키 KEYDOWN 이벤트 발생 시 프로그램을 종료한다.
                elif event.key == K_ESCAPE:
                    terminate()
            elif event.type == KEYUP:
                # p키를 누르면 일시정지.
                if event.key == K_p:
                    DISPLAYSURF.fill(BGCOLOR)
                    # 음악 정지
                    pygame.mixer.music.stop()
                    # Paused 텍스트를 화면에 띄운다.
                    showTextScreen('Paused')
                    # 음악 재실행
                    pygame.mixer.music.play(-1, 0, 0)

        # 지렁이의 머리가 화면 끝을 치는 경우(벽을 치는 경우)
        if wormCoords[HEAD]['x'] == -1 or wormCoords[HEAD]['x'] == CELLWIDTH or wormCoords[HEAD]['y'] == -1 or \
                wormCoords[HEAD]['y'] == CELLHEIGHT:
            return  # game over
        # 지렁이가 머리와 몸이 맞닿는 경우(머리가 몸을 치는 경우)
        for wormBody in wormCoords[1:]:
            if wormBody['x'] == wormCoords[HEAD]['x'] and wormBody['y'] == wormCoords[HEAD]['y']:
                return  # game over

        # 지렁이가 사과를 먹게 되는 경우(머리가 사과에 닿는 경우)
        # 이 경우는 꼬리 부분을 자르지 않음으로써 지렁이의 size를 1만큼 키운다.
        if wormCoords[HEAD]['x'] == apple['x'] and wormCoords[HEAD]['y'] == apple['y']:
            # 새로운 사과를 임의의 위치에 생성한다.
            apple = getRandomLocation()
        else:
            # 지렁이의 꼬리 부분을 제거한다.
            # 이 부분이 없으면 지렁이는 움직일 때마다 그 길이가 1씩 증가한다.
            # 꼬리부분을 자름으로서 지렁이의 길이는 유지된다.
            del wormCoords[-1]

        # 지링이 움직이는 부분.
        # 지렁이의 HEAD 부분을 direction에 따라 수정해준다.
        # 뒤의 몸통 부분은 HEAD가 향하는 방향으로 따라가게 된다.
        if direction == UP:
            newHead = {'x': wormCoords[HEAD]['x'], 'y': wormCoords[HEAD]['y'] - 1}
        elif direction == DOWN:
            newHead = {'x': wormCoords[HEAD]['x'], 'y': wormCoords[HEAD]['y'] + 1}
        elif direction == LEFT:
            newHead = {'x': wormCoords[HEAD]['x'] - 1, 'y': wormCoords[HEAD]['y']}
        elif direction == RIGHT:
            newHead = {'x': wormCoords[HEAD]['x'] + 1, 'y': wormCoords[HEAD]['y']}
        # 지렁이의 새로운 머리를 insert만 해준다.
        wormCoords.insert(0, newHead)
        DISPLAYSURF.fill(BGCOLOR)
        # 격자를 그린다.(while loop가 돌아가는 한 유지된다)
        drawGrid()
        # 지렁이를 화면에 표현
        drawWorm(wormCoords)
        # 사과 화면에 표현
        drawApple(apple)
        # 지렁이의 길이에 따른 score
        # 지렁이의 첫 시작은 길이가 3이므로 -3을 해준다.
        drawScore(len(wormCoords) - 3)
        # Playtime을 구현 및 화면에 띄우는 함수
        # 인자로 기록한 게임의 시작 시각을 넣는다.
        drawTime(start_time)
        pygame.display.update()
        FPSCLOCK.tick(FPS)

# Press a key to play. 를 화면에 띄우는 함수
# press a key to play의 위치를 지정한다.
def drawPressKeyMsg():
    pressKeySurf = BASICFONT.render('Press a key to play.', True, DARKGRAY)
    pressKeyRect = pressKeySurf.get_rect()
    pressKeyRect.topleft = (WINDOWWIDTH - 200, WINDOWHEIGHT - 30)
    DISPLAYSURF.blit(pressKeySurf, pressKeyRect)

# key 입력을 체크하는 함수
def checkForKeyPress():
    if len(pygame.event.get(QUIT)) > 0:
        # 우선 QUIT이 event 큐에 있으면 종료 시킨다.
        terminate()

    keyUpEvents = pygame.event.get(KEYUP)
    # keyUpEvent가 없다면 아무것도 반환하지 않는다.
    if len(keyUpEvents) == 0:
        return None
    # esc키를 누르면 게임을 종료 시킨다.
    if keyUpEvents[0].key == K_ESCAPE:
        terminate()
    # keyup event의 첫번째 key를 반환
    return keyUpEvents[0].key

# 게임 시작 화면을 보여주는 함수
def showStartScreen():
    titleFont = pygame.font.Font('D2Coding.ttc', 100)
    titleSurf1 = titleFont.render('OSW Game', True, BLACK, YELLOW)  # 게임명 변경, 색 변경
    titleSurf2 = titleFont.render('OSW Game', True, YELLOW)  # 게임명, 색 변경

    degrees1 = 0
    degrees2 = 0
    # 이 부분은 "OSW Game"이라는 게임 이미지를 회전 시켜주는 부분이다.
    while True:
        DISPLAYSURF.fill(BGCOLOR)
        rotatedSurf1 = pygame.transform.rotate(titleSurf1, degrees1)
        rotatedRect1 = rotatedSurf1.get_rect()
        rotatedRect1.center = (WINDOWWIDTH / 2, WINDOWHEIGHT / 2)
        DISPLAYSURF.blit(rotatedSurf1, rotatedRect1)

        rotatedSurf2 = pygame.transform.rotate(titleSurf2, degrees2)
        rotatedRect2 = rotatedSurf2.get_rect()
        rotatedRect2.center = (WINDOWWIDTH / 2, WINDOWHEIGHT / 2)
        DISPLAYSURF.blit(rotatedSurf2, rotatedRect2)

        drawPressKeyMsg()

        if checkForKeyPress():
            # esc가 아닌 첫 입력이 발생하면
            # event를 clear하며 시작 화면이 끝난다.
            pygame.event.get()
            return
        pygame.display.update()
        FPSCLOCK.tick(FPS)
        degrees1 += 3  # rotate by 3 degrees each frame
        degrees2 += 7  # rotate by 7 degrees each frame


def makeTextObjs(text, font, color):
    surf = font.render(text, True, color)
    return surf, surf.get_rect()

# 중간에 Paused를 띄울 때 활용하는 함수이다.
def showTextScreen(text):
    # This function displays large text in the
    # center of the screen until a key is pressed.
    # Draw the text
    titleSurf, titleRect = makeTextObjs(text, BASICFONT, TEXTCOLOR)
    titleRect.center = (int(WINDOWWIDTH / 2) - 3, int(WINDOWHEIGHT / 2) - 3)
    DISPLAYSURF.blit(titleSurf, titleRect)

    # Draw the additional "Press a key to play." text.
    pressKeySurf, pressKeyRect = makeTextObjs('Press a key to play.', BASICFONT, TEXTCOLOR)
    pressKeyRect.center = (int(WINDOWWIDTH / 2), int(WINDOWHEIGHT / 2) + 100)
    DISPLAYSURF.blit(pressKeySurf, pressKeyRect)

    while checkForKeyPress() == None:
        # paused를 멈추고 시작하려면 esc를 제외한 key를 눌러야한다
        # key를 누르면 checkForKeyPress()는 반환값이 있으며 while loop는 종료된다.
        pygame.display.update()
        FPSCLOCK.tick()

# 게임을 종료하는 함수.
def terminate():
    pygame.quit()
    sys.exit()

# 사과를 임의의 위치에 배치하는 함수
def getRandomLocation():
    return {'x': random.randint(0, CELLWIDTH - 1), 'y': random.randint(0, CELLHEIGHT - 1)}

# 게임이 끝났음을 화면에 띄우는 함수
def showGameOverScreen():
    # 이 함수의 경우 main 함수 실행 시 runGame() 이후에 실행된다.
    # runGame()이 시작됨에 따라 start_time이라는 변수에 게임의 시작 시각을 저장하였다.
    # 이 start_time은 runGame()이 종료됨에 초기화한다.
    global start_time
    gameOverFont = pygame.font.Font('D2Coding.ttc', 150)
    gameSurf = gameOverFont.render('Game', True, WHITE)
    overSurf = gameOverFont.render('Over', True, WHITE)
    gameRect = gameSurf.get_rect()
    overRect = overSurf.get_rect()
    gameRect.midtop = (WINDOWWIDTH / 2, 10)
    overRect.midtop = (WINDOWWIDTH / 2, gameRect.height + 10 + 25)

    DISPLAYSURF.blit(gameSurf, gameRect)
    DISPLAYSURF.blit(overSurf, overRect)
    drawPressKeyMsg()
    pygame.display.update()
    pygame.time.wait(500)
    checkForKeyPress()  # clear out any key presses in the event queue
    start_time = None  # 새로운 시작 time을 받기 위해서 초기화 시켜준다.

    while True:
        if checkForKeyPress():
            pygame.event.get()  # clear event queue
            return

# score를 화면에 보여주는 함수
def drawScore(score):
    # score는 지렁이의 길이 -3의 값이다.(지렁이의 기본 값 : 3)
    scoreSurf = BASICFONT.render('Score: %s' % (score), True, WHITE)
    scoreRect = scoreSurf.get_rect()
    scoreRect.topleft = (WINDOWWIDTH - 150, 10)
    DISPLAYSURF.blit(scoreSurf, scoreRect)

# Playtime을 화면에 보여주는 함수
def drawTime(start_time):
    # t는 runGame()이 while loop로 돌아가는 중에 계속 시각이 저장된다.
    t = time.time()
    # t - start_time이면 현재 시각과 시작 시각의 차이를 통해 playtime을 구한다.
    # 정수형으로 형 변환을 시켜 초 단위로 표시한다.
    timeSurf = BASICFONT.render('Playtime: %d sec' % int(t - start_time), True, WHITE)
    timeRect = timeSurf.get_rect()
    timeRect.topleft = (WINDOWWIDTH - 150, 40)
    DISPLAYSURF.blit(timeSurf, timeRect)

# 지렁이를 그리는 함수이다.
def drawWorm(wormCoords):
    for coord in wormCoords:
        x = coord['x'] * CELLSIZE
        y = coord['y'] * CELLSIZE
        wormSegmentRect = pygame.Rect(x, y, CELLSIZE, CELLSIZE)
        pygame.draw.rect(DISPLAYSURF, DARKGREEN, wormSegmentRect)
        wormInnerSegmentRect = pygame.Rect(x + 4, y + 4, CELLSIZE - 8, CELLSIZE - 8)
        pygame.draw.rect(DISPLAYSURF, GREEN, wormInnerSegmentRect)

# 사과를 그리는 함수이다.
def drawApple(coord):
    x = coord['x'] * CELLSIZE
    y = coord['y'] * CELLSIZE
    appleRect = pygame.Rect(x, y, CELLSIZE, CELLSIZE)
    pygame.draw.rect(DISPLAYSURF, RED, appleRect)

# 격자판을 그리는 함수이다
def drawGrid():
    # 수직선을 그리는 부분
    for x in range(0, WINDOWWIDTH, CELLSIZE):  # draw vertical lines
        pygame.draw.line(DISPLAYSURF, DARKGRAY, (x, 0), (x, WINDOWHEIGHT))
    # 수평선을 그리는 부분
    for y in range(0, WINDOWHEIGHT, CELLSIZE):  # draw horizontal lines
        pygame.draw.line(DISPLAYSURF, DARKGRAY, (0, y), (WINDOWWIDTH, y))

# main 함수를 실행시키는 part
if __name__ == '__main__':
    main()


pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
